# 데이터 전처리를 실험하는 파일
@creator: 김기범

# Import

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('../dataset/train/train.csv')
test_df = pd.read_csv('../dataset/test/test_data.csv')

train_df.shape, test_df.shape

((32470, 6), (7765, 6))

In [3]:
train_df.head()

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32470 entries, 0 to 32469
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              32470 non-null  int64 
 1   sentence        32470 non-null  object
 2   subject_entity  32470 non-null  object
 3   object_entity   32470 non-null  object
 4   label           32470 non-null  object
 5   source          32470 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.5+ MB


In [5]:
for column in train_df.columns:
    value = train_df.iloc[0][column]
    print(column, ':', value, type(value))

id : 0 <class 'numpy.int64'>
sentence : 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다. <class 'str'>
subject_entity : {'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'} <class 'str'>
object_entity : {'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'} <class 'str'>
label : no_relation <class 'str'>
source : wikipedia <class 'str'>


In [6]:
eval(train_df.iloc[0]['subject_entity'])

{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}

In [7]:
for type_entity in ['subject', 'object']:
    column = f"{type_entity}_entity"

    word_list, type_list = [], []
    start_idx_list, end_idx_list = [], []
    
    for i in range(len(train_df)):
        dictionary = eval(train_df.iloc[i][column])

        word_list.append(dictionary['word'])
        start_idx_list.append(dictionary['start_idx'])
        end_idx_list.append(dictionary['end_idx'])
        type_list.append(dictionary['type'])
    
    train_df[column] = word_list
    for key in ['start_idx', 'end_idx', 'type']:
        train_df[f"{type_entity}_{key}"] = eval(f"{key}_list")

train_df.head()

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,비틀즈,조지 해리슨,no_relation,wikipedia,24,26,ORG,13,18,PER
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,민주평화당,대안신당,no_relation,wikitree,19,23,ORG,14,17,ORG
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,광주FC,한국프로축구연맹,org:member_of,wikitree,21,24,ORG,34,41,ORG
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,아성다이소,박정부,org:top_members/employees,wikitree,13,17,ORG,22,24,PER
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,요미우리 자이언츠,1967,no_relation,wikipedia,22,30,ORG,0,3,DAT
